# DREAM instrument view

This notebook is a simple example of how to use the instrument view for the DREAM instrument.

- The DREAM-specific instrument view is capable of slicing the data with a slider widget along a dimension (e.g. `tof`) by using the `dim` argument.
- There are also checkboxes to hide/show the different elements that make up the DREAM detectors.

We begin with relevant imports.
We will be using tutorial data downloaded with `pooch`.
If you get an error about a missing module `pooch`, you can install it with `!pip install pooch`:

In [ ]:
import scipp as sc
from ess import dream
import ess.dream.data  # noqa: F401

## Load the data

We load a dataset from a Geant4 simulation (stored as a `.csv` file).
In each detector bank, the data is organised by `wire`, `strip`, `module`, `segment`, and `counter`.
The high resolution detector also has an additional `sector` dimension.

In [ ]:
dg = dream.io.load_geant4_csv(dream.data.get_path("data_dream0_new_hkl_Si_pwd.csv.zip"))
dg = dg["instrument"]  # Extract the instrument data

# Extract the events from nested data groups
dg = sc.DataGroup({key: detector["events"] for key, detector in dg.items()})
dg

## Display inhomogeneity from absorption

If the sample absorbs or scatters a large fraction of the incoming neutrons the intensity in the detector will vary even if the scattering is inhomogeneous.


In [ ]:
import scipp as sc

from scippneutron.absorption import compute_transmission_map
from scippneutron.absorption.cylinder import Cylinder
from scippneutron.absorption.material import Material
from scippneutron.atoms import ScatteringParams


material =  Material(
    scattering_params=ScatteringParams.for_isotope('V'),
    effective_sample_number_density=sc.scalar(0.07192, unit='1/angstrom**3')
)
sample_shape = Cylinder(
    symmetry_line=sc.vector([0, 1, 0]),
    center_of_base=sc.vector([0, -2.5, 0], unit='cm'),
    radius=sc.scalar(1, unit='cm'),
    height=sc.scalar(5., unit='cm')
)

transmission_fraction = compute_transmission_map(
    sample_shape,
    material,
    beam_direction=sc.vector([0, 0, 1]),
    wavelength=sc.linspace('wavelength', 4, 8, 20, unit='angstrom'),
    detector_position=dg['mantle'].coords['position']['strip', ::4]['wire', ::2].copy(),
    quadrature_kind='cheap',
)

transmission_fraction.coords['position'] = transmission_fraction.coords.pop('detector_position')
dream.instrument_view(transmission_fraction.transpose((*set(transmission_fraction.dims) - {'wavelength'}, 'wavelength')), dim='wavelength')

In [ ]:
transmission_fraction